In [40]:
 # Dependencies
import requests
import urllib
import random
import math
import pandas as pd
import xml.etree.ElementTree as ET
from config import zws_id, gkey 

In [41]:
# function to grab the exact address based on longitude and latitude
# modified from here https://gist.github.com/bradmontgomery/5397472
# their example didn't include an API key, but I added it otherwise you'd hit the rate limit easily

def reverse_geocode(latitude, longitude):
    # Did the geocoding request comes from a device with a
    # location sensor? Must be either true or false
    sensor = 'true'

    # Hit Google's reverse geocoder directly
    # NOTE: I *think* their terms state that you're supposed to
    # use google maps if you use their api for anything.
    base = "https://maps.googleapis.com/maps/api/geocode/json?"
    params = "latlng={lat},{lon}&sensor={sen}&key={key}".format(
        lat=latitude,
        lon=longitude,
        sen=sensor,
        key=gkey
    )
    url = "{base}{params}".format(base=base, params=params)
    #print(url)
    response = requests.get(url).json()
    address = response['results'][0]['formatted_address']
    return address


In [28]:
# function to generate random lat & lng within a certain radius 
# modified from here: http://hadoopguru.blogspot.com/2014/12/python-generate-random-latitude-and.html
# changed to take in an empty initial dataframe and load in the data + return it
# this calls the reverse geocode function to grab the addresses of each randomly generated lat & lng

def generate_latlng(latitude, longitude, df):
    
    radius = 10000                         #Choose your own radius
    radiusInDegrees=radius/111300            
    r = radiusInDegrees

    counter = 0
    
    for i in range(1,100):                 #Choose number of Lat Long to be generated

        u = float(random.uniform(0.0,1.0))
        v = float(random.uniform(0.0,1.0))

        w = r * math.sqrt(u)
        t = 2 * math.pi * v
        x = w * math.cos(t) 
        y = w * math.sin(t)

        xLat  = x + latitude
        yLng = y + longitude

        df.set_value(counter, "latitude", xLat)
        df.set_value(counter, "longitude", yLng)
        
        #print(format(counter) + ": " + format(xLat) + ", " + format(yLng))
        address = reverse_geocode(xLat, yLng).split(',')
        citystatezip = address[1] + address[2]
        
        df.set_value(counter, "address", address[0])
        df.set_value(counter, "city_state_zip", citystatezip)
        
        # Add to counter
        counter = counter + 1
    
    return df

# use Los Angeles as the center of the Latitude & longitude to test this out
# if instructor allows, we should probably just manually call this 20 times to get data for each city faster
# otherwise the gigantic loop will take forever

la_df = pd.DataFrame({"latitude":'',
                   "longitude":'',
                   "address":'',
                   "city_state_zip":''}, index=[0])

generate_latlng(34.0522342,-118.2436849, la_df)

,address,city_state_zip,latitude,longitude
0,234 S Rowan Ave,Los Angeles CA 90063,34.0335,-118.187
1,514 S Chevy Chase Dr,Glendale CA 91205,34.1402,-118.239
2,3228 Rowena Ave,Los Angeles CA 90027,34.1104,-118.275
3,3124-3126 N Eastern Ave,Los Angeles CA 90032,34.0817,-118.178
4,2813 Silver St,Los Angeles CA 90065,34.1006,-118.234
5,5771 Omaha St,Los Angeles CA 90042,34.1057,-118.19
6,310-312 Boyd St,Los Angeles CA 90013,34.0471,-118.244
7,3719 Verdugo Rd,Los Angeles CA 90065,34.1177,-118.233
8,3719 Roderick Rd,Los Angeles CA 90065,34.1199,-118.235
9,116N N Irving Blvd,Los Angeles CA 90004,34.0737,-118.318


In [42]:
### UNFINISHED CODE FROM THIS POINT ON ###
### STILL BEING WORKED ON ###

# TO-DO: handle error code where house does not exist at that address, skip over and/or cull from table

# zillow url format
# http://www.zillow.com/webservice/GetSearchResults.htm?zws-id=<ZWSID>&address=2114+Bigelow+Ave&citystatezip=Seattle%2C+WA

url = 'http://www.zillow.com/webservice/GetSearchResults.htm?zws-id='

url_address = la_df['address'][1]
url_citystatezip = la_df['city_state_zip'][1]

query_url = url + zws_id + '&address=' + urllib.parse.quote(url_address) + '&citystatezip=' + urllib.parse.quote(url_citystatezip) 
print(query_url)

http://www.zillow.com/webservice/GetSearchResults.htm?zws-id=X1-ZWz1gba2odrvgr_6plsv&address=514%20S%20Chevy%20Chase%20Dr&citystatezip=%20Glendale%20CA%2091205


In [38]:
# traverse xml to pull certain pieces of data out

root = ET.parse(urllib.request.urlopen(query_url)).getroot()

# grab the zestimate
for zestimate in root.iter('zestimate'):
    print (zestimate[0].text)

# grab the difference in valuation
for valuation in root.iter('valuationRange'):
    lowValuation = int(valuation[0].text)
    highValuation = int(valuation[1].text)
    valuationRange = highValuation - lowValuation
    print(valuationRange)
    #print(lowValuation)
    #print(highValuation)

# grab the following
# address
# price 
# valuation range

822934
82294
